## Exploring Ebay Car Sales Data
`dateCrawled` - *When this ad was first crawled. All field-values are taken from this date*.

`name` - *Name of the car*.

`seller` - *Whether the seller is private or a dealer*.

`offerType` - *The type of listing*

`price` - *The price on the ad to sell the car*.

`abtest` - *Whether the listing is included in an A/B test*.

`vehicleType` - *The vehicle Type*.

`yearOfRegistration` - *The year in which which year the car was first registered*.

`gearbox` - *The transmission type*.

`powerPS` - *The power of the car in PS*.

`model` - *The car model name*.

`kilometer` - *How many kilometers the car has driven*.

`monthOfRegistration` - *The month in which which year the car was first registered*.

`fuelType` - *What type of fuel the car uses*.

`brand` - *The brand of the car*.

`notRepairedDamage` - *If the car has a damage which is not yet repaired*.

`dateCreated` - *The date on which the eBay listing was created*.

`nrOfPictures` - *The number of pictures in the ad*.

`postalCode` - *The postal code for the location of the vehicle*.

`lastSeenOnline` - *When the crawler saw this ad last online*.

In [2]:
import numpy as np
import pandas as pd
import datetime

autos = pd.read_csv("autos.csv", encoding="Latin-1")

### Checking data for null values for cleaning

In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

### Converting the column names from camelcase to snakecase

In [4]:
autos["registration_year"] = autos["yearOfRegistration"]
autos["registration_month"] = autos["monthOfRegistration"]
autos["unrepaired_damage"] = autos["notRepairedDamage"]
autos["ad_created"] = autos["dateCreated"]
autos["ad_crawled"] = autos["dateCrawled"]
autos["type_offer"] = autos["offerType"]
autos["type_vehicle"] = autos["vehicleType"]
autos["power_PS"] = autos["powerPS"]
autos["type_fuel"] = autos["fuelType"]
autos["picture_num"] = autos["nrOfPictures"]
autos["postal_code"] = autos["postalCode"]
autos["last_online"] = autos["lastSeen"]

autos.drop(["yearOfRegistration","monthOfRegistration","notRepairedDamage", "dateCreated", "dateCrawled","offerType","vehicleType","powerPS","fuelType","nrOfPictures","postalCode","lastSeen"], axis=1, inplace=True)
autos.columns = [c.strip() for c in autos.columns]

autos.info()
autos.head()
autos.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
name                  50000 non-null object
seller                50000 non-null object
price                 50000 non-null object
abtest                50000 non-null object
gearbox               47320 non-null object
model                 47242 non-null object
odometer              50000 non-null object
brand                 50000 non-null object
registration_year     50000 non-null int64
registration_month    50000 non-null int64
unrepaired_damage     40171 non-null object
ad_created            50000 non-null object
ad_crawled            50000 non-null object
type_offer            50000 non-null object
type_vehicle          44905 non-null object
power_PS              50000 non-null int64
type_fuel             45518 non-null object
picture_num           50000 non-null int64
postal_code           50000 non-null int64
last_online           50000 non-null object
dtypes: int64(5)

Index(['name', 'seller', 'price', 'abtest', 'gearbox', 'model', 'odometer',
       'brand', 'registration_year', 'registration_month', 'unrepaired_damage',
       'ad_created', 'ad_crawled', 'type_offer', 'type_vehicle', 'power_PS',
       'type_fuel', 'picture_num', 'postal_code', 'last_online'],
      dtype='object')

From column "`price`" we can delete character '$' and convert to integer.

"`odometer`" - delete 'km' and convert to integer.

"`ad_created`", "`ad_crawled`", "`last_online`" - convert to datetime.

"`gearbox`", "`model`", "`unrepaired_damage`", "`type_vehicle`", "`type_fuel`" - need more investigation.

In [5]:
autos.describe(include='all')

,name,seller,price,abtest,gearbox,model,odometer,brand,registration_year,registration_month,unrepaired_damage,ad_created,ad_crawled,type_offer,type_vehicle,power_PS,type_fuel,picture_num,postal_code,last_online
count,50000,50000,50000,50000,47320,47242,50000,50000,50000.000000,50000.000000,40171,50000,50000,50000,44905,50000.000000,45518,50000.0,50000.000000,50000
unique,38754,2,2357,2,2,245,13,40,NaN,NaN,2,76,48213,2,8,NaN,7,NaN,NaN,39481
top,Ford_Fiesta,privat,$0,test,manuell,golf,"150,000km",volkswagen,NaN,NaN,nein,2016-04-03 00:00:00,2016-03-22 09:51:06,Angebot,limousine,NaN,benzin,NaN,NaN,2016-04-07 06:17:27
freq,78,49999,1421,25756,36993,4024,32424,10687,NaN,NaN,35232,1946,3,49999,12859,NaN,30107,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,5.723360,NaN,NaN,NaN,NaN,NaN,116.355920,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,3.711984,NaN,NaN,NaN,NaN,NaN,209.216627,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,3.000000,NaN,NaN,NaN,NaN,NaN,70.000000,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,6.000000,NaN,NaN,NaN,NaN,NaN,105.000000,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,9.000000,NaN,NaN,NaN,NaN,NaN,150.000000,NaN,0.0,71540.000000,NaN


In [6]:
autos["price"] = (autos["price"]
                          .str.replace("$","")
                          .str.replace(",","")
                          .astype(int)
                          )
autos["odometer"] = (autos["odometer"]
                         .str.replace("km","")
                         .str.replace(",","")
                         .astype(int)
                        )
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)

### `odometer_km` and `price` info

In [7]:
for col in autos["odometer_km"], autos["price"]:
    print(col.unique().shape)
    print(col.describe())
    print(col.value_counts().head(10))

(13,)
count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64
150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
Name: odometer_km, dtype: int64
(2357,)
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64
0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
Name: price, dtype: int64


In [9]:
(autos["ad_crawled"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

2016-03-05    0.02538
2016-03-06    0.01394
2016-03-07    0.03596
2016-03-08    0.03330
2016-03-09    0.03322
2016-03-10    0.03212
2016-03-11    0.03248
2016-03-12    0.03678
2016-03-13    0.01556
2016-03-14    0.03662
2016-03-15    0.03398
2016-03-16    0.02950
2016-03-17    0.03152
2016-03-18    0.01306
2016-03-19    0.03490
2016-03-20    0.03782
2016-03-21    0.03752
2016-03-22    0.03294
2016-03-23    0.03238
2016-03-24    0.02910
2016-03-25    0.03174
2016-03-26    0.03248
2016-03-27    0.03104
2016-03-28    0.03484
2016-03-29    0.03418
2016-03-30    0.03362
2016-03-31    0.03192
2016-04-01    0.03380
2016-04-02    0.03540
2016-04-03    0.03868
2016-04-04    0.03652
2016-04-05    0.01310
2016-04-06    0.00318
2016-04-07    0.00142
Name: ad_crawled, dtype: float64

In [8]:
(autos["ad_crawled"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values()
        )

2016-04-07    0.00142
2016-04-06    0.00318
2016-03-18    0.01306
2016-04-05    0.01310
2016-03-06    0.01394
2016-03-13    0.01556
2016-03-05    0.02538
2016-03-24    0.02910
2016-03-16    0.02950
2016-03-27    0.03104
2016-03-17    0.03152
2016-03-25    0.03174
2016-03-31    0.03192
2016-03-10    0.03212
2016-03-23    0.03238
2016-03-26    0.03248
2016-03-11    0.03248
2016-03-22    0.03294
2016-03-09    0.03322
2016-03-08    0.03330
2016-03-30    0.03362
2016-04-01    0.03380
2016-03-15    0.03398
2016-03-29    0.03418
2016-03-28    0.03484
2016-03-19    0.03490
2016-04-02    0.03540
2016-03-07    0.03596
2016-04-04    0.03652
2016-03-14    0.03662
2016-03-12    0.03678
2016-03-21    0.03752
2016-03-20    0.03782
2016-04-03    0.03868
Name: ad_crawled, dtype: float64

Site was viewed every day from April to March about the same.

In [9]:
(autos["ad_created"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

2015-06-11    0.00002
2015-08-10    0.00002
2015-09-09    0.00002
2015-11-10    0.00002
2015-12-05    0.00002
2015-12-30    0.00002
2016-01-03    0.00002
2016-01-07    0.00002
2016-01-10    0.00004
2016-01-13    0.00002
2016-01-14    0.00002
2016-01-16    0.00002
2016-01-22    0.00002
2016-01-27    0.00006
2016-01-29    0.00002
2016-02-01    0.00002
2016-02-02    0.00004
2016-02-05    0.00004
2016-02-07    0.00002
2016-02-08    0.00002
2016-02-09    0.00004
2016-02-11    0.00002
2016-02-12    0.00006
2016-02-14    0.00004
2016-02-16    0.00002
2016-02-17    0.00002
2016-02-18    0.00004
2016-02-19    0.00006
2016-02-20    0.00004
2016-02-21    0.00006
               ...   
2016-03-09    0.03324
2016-03-10    0.03186
2016-03-11    0.03278
2016-03-12    0.03662
2016-03-13    0.01692
2016-03-14    0.03522
2016-03-15    0.03374
2016-03-16    0.03000
2016-03-17    0.03120
2016-03-18    0.01372
2016-03-19    0.03384
2016-03-20    0.03786
2016-03-21    0.03772
2016-03-22    0.03280
2016-03-23

In [10]:
(autos["ad_created"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values()
        )

2016-02-16    0.00002
2016-01-13    0.00002
2016-02-22    0.00002
2016-01-29    0.00002
2016-02-11    0.00002
2016-02-07    0.00002
2016-01-14    0.00002
2016-01-22    0.00002
2016-02-08    0.00002
2016-02-01    0.00002
2015-11-10    0.00002
2016-01-07    0.00002
2015-06-11    0.00002
2015-09-09    0.00002
2016-02-17    0.00002
2016-01-03    0.00002
2016-01-16    0.00002
2015-08-10    0.00002
2015-12-05    0.00002
2015-12-30    0.00002
2016-02-24    0.00004
2016-02-14    0.00004
2016-02-02    0.00004
2016-02-09    0.00004
2016-02-05    0.00004
2016-02-26    0.00004
2016-02-18    0.00004
2016-01-10    0.00004
2016-02-20    0.00004
2016-02-25    0.00006
               ...   
2016-03-06    0.01512
2016-03-13    0.01692
2016-03-05    0.02304
2016-03-24    0.02908
2016-03-16    0.03000
2016-03-27    0.03090
2016-03-17    0.03120
2016-03-10    0.03186
2016-03-25    0.03188
2016-03-31    0.03192
2016-03-23    0.03218
2016-03-26    0.03256
2016-03-11    0.03278
2016-03-22    0.03280
2016-03-09

Fresher ads have more views.

In [11]:
(autos["last_online"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values()
        )

2016-03-05    0.00108
2016-03-06    0.00442
2016-03-07    0.00536
2016-03-18    0.00742
2016-03-08    0.00760
2016-03-13    0.00898
2016-03-09    0.00986
2016-03-10    0.01076
2016-03-11    0.01252
2016-03-14    0.01280
2016-03-19    0.01574
2016-03-15    0.01588
2016-03-27    0.01602
2016-03-16    0.01644
2016-03-26    0.01696
2016-03-23    0.01858
2016-03-25    0.01920
2016-03-24    0.01956
2016-03-20    0.02070
2016-03-21    0.02074
2016-03-28    0.02086
2016-03-22    0.02158
2016-03-29    0.02234
2016-04-01    0.02310
2016-03-12    0.02382
2016-03-31    0.02384
2016-04-04    0.02462
2016-03-30    0.02484
2016-04-02    0.02490
2016-04-03    0.02536
2016-03-17    0.02792
2016-04-05    0.12428
2016-04-07    0.13092
2016-04-06    0.22100
Name: last_online, dtype: float64

The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales.

In [12]:
autos["registration_year"].describe()

count    50000.000000
mean      2005.073280
std        105.712813
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

The year when the car was first registered was likely to be the year of release. The minimum value of "1000" is not reliable because the machines were invented later, the maximum value - "9999" - exceeds the current year.

Because a car can't be first registered before the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

In [13]:
autos = autos.loc[(autos["registration_year"] > 1900) & (autos["registration_year"] < 2006)]
autos["registration_year"].value_counts(normalize=True)

2000    0.108382
2005    0.097428
1999    0.096943
2004    0.088444
2003    0.088121
2001    0.087346
2002    0.081852
1998    0.079267
1997    0.065534
1996    0.046662
1995    0.042429
1994    0.021327
1993    0.014380
1990    0.012764
1992    0.012635
1991    0.011504
1989    0.005849
1988    0.004589
1985    0.003393
1980    0.003134
1986    0.002456
1987    0.002424
1983    0.001713
1984    0.001713
1978    0.001519
1970    0.001454
1982    0.001390
1972    0.001131
1979    0.001131
1960    0.001099
          ...   
1966    0.000711
1977    0.000711
1975    0.000614
1969    0.000614
1965    0.000549
1964    0.000388
1963    0.000291
1910    0.000291
1959    0.000226
1961    0.000194
1956    0.000162
1958    0.000129
1962    0.000129
1937    0.000129
1950    0.000097
1951    0.000065
1941    0.000065
1955    0.000065
1957    0.000065
1934    0.000065
1954    0.000065
1953    0.000032
1943    0.000032
1938    0.000032
1939    0.000032
1927    0.000032
1929    0.000032
1931    0.0000

Most of cars were registered in the past 20 years.

In [14]:
brand_counts = autos["brand"].value_counts(normalize=True).head(7)
brands = brand_counts.index
brands

Index(['volkswagen', 'opel', 'bmw', 'mercedes_benz', 'ford', 'audi',
       'renault'],
      dtype='object')

We will explore the most popular cars.

In [16]:
brand_mean_price = {}

for b in brands:
    selected_rows = autos[autos["brand"] == b]
    mean_price = selected_rows["price"].mean()
    brand_mean_price[b] = int(mean_price)
    
brand_mean_price

{'audi': 3497,
 'bmw': 4561,
 'ford': 6754,
 'mercedes_benz': 36700,
 'opel': 4798,
 'renault': 1335,
 'volkswagen': 4445}

The most expensive brand is "Mercedes Benz".

The cheapest brand is "Renault".

### Brand info into DataFrame

In [18]:
bmp_series = pd.Series(brand_mean_price)
brand_info = pd.DataFrame(bmp_series, columns=['mean_price'])
brand_info

,mean_price
audi,3497
bmw,4561
ford,6754
mercedes_benz,36700
opel,4798
renault,1335
volkswagen,4445


In [21]:
brand_mean_mileage = {}

for b in brands:
    selected_rows = autos[autos["brand"] == b]
    mean_mileage = selected_rows["odometer_km"].mean()
    brand_mean_mileage[b] = int(mean_mileage)
    
brand_mean_mileage

{'audi': 144415,
 'bmw': 141977,
 'ford': 132548,
 'mercedes_benz': 139709,
 'opel': 136660,
 'renault': 136841,
 'volkswagen': 139579}

In [24]:
bmm_series = pd.Series(brand_mean_mileage)
brand_info["mean_mileage"] = bmm_series
brand_info.sort_values(by=['mean_mileage'], ascending=False)
brand_info

,mean_price,mean_mileage
audi,3497,144415
bmw,4561,141977
ford,6754,132548
mercedes_benz,36700,139709
opel,4798,136660
renault,1335,136841
volkswagen,4445,139579


There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.